# AIAP Foundation Self Practice

The objective is to predict the students' O-level mathematics examination score to help the school to identify weaker students prior to the examination using the dataset provided. In your submission, you should evaluate at least 3 suitable models for estimating the students' scores.

## Data Dictionary

| Column               | Description                        |
| -------------------- | ---------------------------------- |
| student_id           | Unique ID for each student         |
| number_of_siblings   | Number of siblings                 |
| direct_admission     | Mode of entering the school        |
| CCA                  | Enrolled CCA                       |
| learning_style       | Primary learning style             |
| tuition              | Indication of whether the student has a tuition   |
| final_test           | Student's O-level mathematics examination score   |
| n_male               | Number of male classmates          |
| n_female             | Number of female classmates        |
| gender               | Gender type                        |
| age                  | Age of the student                 |
| hours_per_week       | Number of hours student studies per week          |
| attendance_rate      | Attendance rate of the student (%) |
| sleep_time           | Daily sleeping time (hour:minutes) |
| wake_time            | Daily waking up time (hour:minutes)               |
| mode_of_transport    | Mode of transport to school        |
| bag_color            | Colours of student's bag           |

<br>
<hr>

## Exploratory Data Analysis

### Load libraries

In [59]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import pprint
from pathlib import Path

# Load third party libraries
from app_logging.app_logging import Logger
Logger.initialize()
Logger.debug('Loaded libraries.')

### Setting notebook settings

In [ ]:
# Setting Jupyter notebook settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 999)

Logger.debug('Starting EDA.')

In [ ]:
# Setting global variables for dataset path and path to save plots and graphs
FILE_PATH = "./data/regression_bonus_practice_data.csv"
PLOT_PATH = "./images/"

### Load data

In [ ]:
try:
    print(f"Attempting to read the file at: {FILE_PATH}")
    # The main attempt to read the CSV file
    df = pd.read_csv(FILE_PATH)    
    print("\n✅ File read successfully!")
    print("Here are the first 5 rows:")
    print(f"Dataset read has {df.shape[0]:,} rows by {df.shape[1]:,} columns.")
    print(df.head())

except FileNotFoundError:
    print(f"\n❌ ERROR: The file at '{FILE_PATH}' was not found.")
    print("Please make sure the file exists and the path is correct.")
    sys.exit(1) # Stop the program with an error code

except pd.errors.ParserError:
    print(f"\n❌ ERROR: The file at '{FILE_PATH}' is not a valid CSV.")
    print("Please check that the file is a standard, comma-separated text file.")
    sys.exit(1) # Stop the program with an error code
    
except Exception as e:
    # Catch any other unexpected errors
    print(f"\n❌ An unexpected error occurred: {e}")
    sys.exit(1)


### Checking dataset structure

In [ ]:
# Displaying the dataset structure and data types
df.info()

### Checking for missing values

In [ ]:
# Identify columns with missing data
missing_counts = df.isnull().sum()
missing_percent = df.isnull().mean() * 100

if missing_counts.sum() > 0:
    # When there are missing values

    # Filter columns with missing values
    missing_columns = missing_counts[missing_counts > 0].index.tolist()
    Logger.debug(f'Columns with missing values: {missing_columns}')

    # Display columns with missing values, their count, and percentage
    print("Columns with missing data:\n")
    for col in missing_columns:
        print(f"{col:<25} : {missing_counts[col]:>6,} missing ({missing_percent[col]:>5.2f}%)")
        Logger.debug(f"{col:<25} : {missing_counts[col]:>6,} missing ({missing_percent[col]:>5.2f}%)")
else:
    # When there are no missing values
    Logger.info('There are no missing values in the dataset.')

### Checking numerical columns statistics

In [ ]:
# Displaying transposed output of numerical columns statistics
df.describe().T

### Checking for duplicates

In [ ]:
# Checking for duplicates
if df.duplicated().sum() == 0:
    # When there are no duplicatres in the dataset
    # print('There are no duplicates in the dataset.')
    Logger.info('There are no duplicates in the dataset.')
else:
    # When there are duplicates found in the dataset
    print('There are duplicates found in the dataset.')
    print(f"There are a total of {len(df.duplcated().sum):,} duplicates.")
    print('Recommend dropping the duplicates.')
    Logger.debug(f'There are {len(df.duplcated().sum):,} duplicate records found.')

### Checking non-numerical columns statistics

In [ ]:
# Checking object type columns
# df.select_dtypes(include = 'object').value_counts()
Logger.info('Listing non-numerical columns statistics.')
value_counts_dict = {col: df[col].value_counts() for col in df.select_dtypes(include='object').columns}

print('Breakdown of the frequency of values in the object data type.')
print('-' * 60)
pprint.pprint(value_counts_dict)


## Data Cleaning

In [ ]:
Logger.debug('Starting the data cleaning phase.')

### Cleaning `CCA` column

In [ ]:
# Displaying the breakdown of the CCA activities students have joined
display(df['CCA'].value_counts())

print(f"Total missing values in the column: {df['CCA'].isna().sum():,}")

As there are duplicates in the CCA values, we need to standardize on the values but converting them to upper case.

Also, as there are missing values in the column, it will be set to 'NONE'.

In [ ]:
# Converting all values to uppercase
df['CCA'] = df['CCA'].str.upper()
Logger.debug('Converting to uppercase all values in CCA column.')

# Filling the missing values with the 'NONE' category
df['CCA'] = df['CCA'].fillna('NONE')
Logger.debug('Filling all blank values in CCA to NONE.')

# Display the breakdown again
display(df['CCA'].value_counts())

### Creating sleep duration column

The 2 columns - **sleep_time** and **wake_time** - seem to be stgoring the times the students sleep and awake for school. in HH:MM format. These 2 columns will be converted into datetime formaats. Then, an arithmatic operation will be performed to calculate the sleep duration of the student. Then, it will be converted into minutes.

In [ ]:
# Convert the 2 string columns to datetime objects
df['sleep_time_dt'] = pd.to_datetime(df['sleep_time'], format='%H:%M')
df['wake_time_dt'] = pd.to_datetime(df['wake_time'], format='%H:%M')
Logger.debug('Converted sleep_time and wake_time columns to datetime formats.')

# Calculate the duration, not forgetting to handle the overnight timings
duration = np.where(
    df['wake_time_dt'] < df['sleep_time_dt'],
    # If wake time is "before" sleep time, add a day to wake time
    df['wake_time_dt'] + pd.Timedelta(days=1) - df['sleep_time_dt'],
    # Otherwise, it's a simple subtraction
    df['wake_time_dt'] - df['sleep_time_dt']
)
df['sleep_duration'] = duration

# Convert the duration (Timedelta) to total minutes
df['sleep_minutes'] = (df['sleep_duration'].dt.total_seconds() / 60).astype(int)
Logger.debug('Created sleep_duration feature in minutes.')

# Display the column breakdown
display(df['sleep_minutes'].describe())

In [ ]:
# Display the sleep minutes frequency breakdown
df['sleep_minutes'].value_counts()

On the whole, more than 90% of students sleep at least 8 hours of sleep. in fact about 2.4% of the students do not get sufficient sleep, and they sleep at least 5 hours. Will need to investigate if sleep time impacts a student's final test result.

### Cleaning `tuition` column

In [ ]:
df['tuition'].value_counts()

There are inconsistencies in the values encoded into this column. There are 4 values for whether a sudent receives tuition or not. As such, we will standardize the encoding to only 2 values - 'Y' and 'N'. 

We will need to check if a model can differentiate if the student receiving higher marks received tuition.

In [ ]:
# Create replacement mapping values
tuition_replacement_code = {'Yes': 'Y', 
                            'No': 'N'
                            }
# Perform the replacement operationg
df['tuition'] = df['tuition'].replace(tuition_replacement_code)
Logger.debug('Standardized tuition column to 2 values.')

# Verify that the replacement is successful
df['tuition'].value_counts()

### Filling up missing values in `attendance_rate`

In [ ]:
display(df['attendance_rate'].describe())

print(f"Total number of missing values: {df['attendance_rate'].isna().sum():,}")
# print(len(df['attendance_rate'].isna().sum()))

As there are missing values in this column, about 778 observations, or 4.48%, we will use the median value of this column, which is at 95%.

In [ ]:
# Calculate the median value of the attendance_rate column
attendance_rate_median = df['attendance_rate'].median()

# Assign the missing values with the calculated median 
df['attendance_rate'] = df['attendance_rate'].fillna(attendance_rate_median)
print('Missing values in attendance rate has been replaced with the median.')
Logger.debug('Filled missing values in attendance_rate column witrh median.')

# df['attendance_rate'].describe()

# Checking if ther are any empty values in the CCQ column
if df['attendance_rate'].isna().sum() == 0:
    # No more missing values in the column
    print('There are no more empty values.')
    Logger.debug('There are no more missing values in the attendance_rate column.')
else:
    # There are still missing values
    print('NOTE: There are still missing values in the column.')
    print('This requires your attention')
    Logger.debug('There are missing values in the attendance_rate column.')
    Logger.debug(f"Missing values in attendance_rate: {df['attendance_rate'].isna().sum()}")
    sys.exit(1)    

### Cleaning `final_test` column

This column has missing values, and will also be replaced with the median value.

In [ ]:
# Calculating the median value of the 'final_test' column
final_test_median = df['final_test'].median()
print(f"The median value of the column {final_test_median}")

# Replacing the missing value with the median value
df['final_test'].fillna(final_test_median, inplace = True)
Logger.debug(f'Filling missing values in final_test column with median value {final_test_median}.')

The missing value in the `final_test` column has been replaced with the median value of the column, which is 68.0.

### Checking `age` distribution

In [ ]:
df['age'].describe()

In [ ]:
df['age'].value_counts()

From the frequency table above, there are several erroneous values in the `age` column. Thus, I would perform the following operations to rectify the problem:

1. Take the absolute value of their ages.
2. Create a replacement age map so that single digit values are converted into the teens (i.e., 14, 15, 16)
3. Instead of leaving the single value of '14' as a standalone age value, I will change it to 15, as most students who sit for the 'O' level examinations are usually in their 15s and 16s. 

In [ ]:
df['age'] = df['age'].abs()

age_replacement_map = {4: 15,
                       5: 15,
                       6: 16
                       }

# Perform the replacement operationg
df['age'] = df['age'].replace(age_replacement_map)
Logger.debug('Corrected erroneous values in the age column.')

# Verify that the replacement is successful
df['age'].value_counts()

### Checking if missing value still persists

In [ ]:
# print(df.isnull().sum().sum())

if df.isnull().sum().sum() == 0:
    print('All missing values have been treated.')
    Logger.debug('All missing values have been treated.')
else:
    print('There are still missing values in the dataset.')
    Logger.debug('There are still missing values in the dataset.')
    sys.exit(1)

In [ ]:
# Displaying the top 10 records of the post-cleaning
df.head(10)

### Saving a clean copy

In [ ]:
# df_null_score = df[df['final_test'].isnull()]

# df.dropna(inplace = True)

# Creating a duplicate copy of the dataset post cleanup
df_cleaned = df.copy(deep = True)

df_cleaned.to_csv('./data/cleaned_data.csv')
Logger.debug('Saved a clean copy to data folder.')

### Dropping non-essential columns

In [ ]:
# Removing non-essential colukns
columns_to_drop = ['index', 'student_id', 'sleep_time', 'wake_time', 'sleep_time_dt', 'wake_time_dt', 'sleep_duration']
try:
        df.drop(columns = columns_to_drop, 
                inplace = True)
        Logger.debug(f'Dropped non-essential columns: {columns_to_drop}')
except KeyError:
        Logger.warning(f'Columns {columns_to_drop} not found.')

### Checking cleaned data structure

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

## Univariate analysis

In [ ]:
Logger.debug('Starting univariate analysis.')

### Pie Charts

In [ ]:
# Creating a list of columns
columns_to_plot = [
    'number_of_siblings', 'direct_admission', 'learning_style', 
    'gender', 'mode_of_transport', 'bag_color'
]

# Create a figure and a set of subplots, a 2x3 grid of pie charts
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 12))
fig.suptitle('Distribution of Categorical Features', fontsize=20)

# Flatten the axes array to make it easy to loop over
axes = axes.flatten()

# Loop through columns and plot on each subplot
for i, column in enumerate(columns_to_plot):
    value_counts = df[column].value_counts()
    ax = axes[i] # Select the subplot
    
    ax.pie(
        value_counts, 
        labels=value_counts.index, 
        autopct='%1.1f%%', 
        startangle=90,
        # Add some styling for a cleaner look
        wedgeprops={'edgecolor': 'white'},
        textprops={'fontsize': 18} 
    )
    ax.set_title(f'{column.replace("_", " ").title()}', fontsize = 20)

# If you have an odd number of plots, you might want to hide the last empty one
# for i in range(len(columns_to_plot), len(axes)):
#     fig.delaxes(axes[i])

plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make room for the suptitle
plt.savefig(PLOT_PATH + "pie_charts_6.png")
plt.show()

#### Analysis notes

- **Number of siblings**: 42.2% of the observations in the cleaned dataset haws 1 other siblings, followed by 34.6% has no other siblings, while 23.3% has 2 siblings. It is possible that having siblings may lead to a higher test scores.
- **Direct admission**: Nearly 3/4, or exactly 70.5% of the students did not get admitted directly.
- **Learning style**: 57.4% of students learn through auditory methods, which means they acquire retain knowledge better listening to lectures and podcasts/videos. We would need to check students who learn through auditory or visual learning style.
- **Gender**: Students are equally balanced between male and female.
- **Mode of transport**: 
- **Bag color**: The color of the students' bags seems to be fairly distributed for all 6 colors. But does the choice of bag color affect a student's mathematics score?
  

### Histogram of `CCA`

In [ ]:
# 2. Create the histogram using Matplotlib
plt.figure(figsize=(10, 6)) # Set the figure size for better readability

plt.hist(
    df['CCA'], 
    bins=10,          # You can adjust the number of bins to see more or less detail
    color='skyblue',  # Set the color of the bars
    edgecolor='black' # Add black edges to bars for better separation
)

# 3. Add labels and a title for clarity
plt.title('Distribution of CCA Participation', fontsize=16)
plt.xlabel('CCA Participation', fontsize=12)
plt.ylabel('Frequency (Number of Students)', fontsize=12)
plt.grid(axis='y', alpha=0.75) # Add a grid for the y-axis

# SDave the plot
plt.savefig(PLOT_PATH + "bar_chart_CCA.png")

# 4. Display the plot
plt.show()


#### Analysis notes

The initial dataset has about 25% of observationds having missing values.

### Histogram of `final_test'

In [ ]:
# Set a visually appealing style for the plot
sns.set_theme(style="darkgrid")

# Create a figure and axes for the plot. This allows for size adjustments.
plt.figure(figsize=(10, 6))

# Generate the histogram using seaborn's histplot
# 'kde=True' adds a smooth line to estimate the distribution's shape
ax = sns.histplot(data=df, x='final_test', bins=15, kde=True)  ## color='skyblue')

# --- 3. Customizing the Plot ---

# Add a title and labels for clarity and professionalism
ax.set_title('Distribution of Final Test Scores') ## , fontsize=18, pad=20)
ax.set_xlabel('Final Test Score')  ##, fontsize=14)
ax.set_ylabel('Number of Students (Frequency)') ##, fontsize=14)

# You can even add a vertical line for the mean score
mean_score = df['final_test'].mean()
ax.axvline(mean_score, color='red', linestyle='--', linewidth=2, label=f'Mean Score: {mean_score:.2f}')
ax.legend() # Display the label for the mean line

plt.savefig(PLOT_PATH + 'histogram_CCA.png')

# Display the final plot
plt.show()

### Boxplot of `attendance_rate` and `final_test`

In [ ]:
# Set the style for a cleaner look
sns.set_style("darkgrid")

# Create a figure and a set of subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(10, 6))

# Plot for Attendance Rate
sns.boxplot(ax=axes[0], y=df['attendance_rate'], color='skyblue')
axes[0].set_title('Distribution of Attendance Rate') ##, fontsize=14)
axes[0].set_ylabel('Rate (%)')
axes[0].set_xlabel('') # Hide x-axis label as it's not needed

# Plot for Final Test
sns.boxplot(ax=axes[1], y=df['final_test'], color='lightgreen')
axes[1].set_title('Distribution of Final Test Scores')  ##, fontsize=14)
axes[1].set_ylabel('Score (%)')
axes[1].set_xlabel('') # Hide x-axis label

# Add a main title for the entire figure
plt.suptitle('Boxplot of Attendance Rate and Final Test Score', fontsize=16, y=1.02)

# Adjust layout to prevent titles from overlapping
plt.tight_layout()

plt.savefig(PLOT_PATH + 'boxplot.png')

# Display the plot
plt.show()

From the above charts, there are a number of outliers in the attendance rate, but not for the marks for the Mathematics test. I will investigate on the quantum of students whose attendance is lower than the lower bound of the boxplot and whether there is any correlation with their final marks.

#### Investigate the `attendance_rate` column

In [ ]:
# Define the column under investigation
col = 'attendance_rate'

# print(df[col].quantile(0.25))
# print(df[col].quantile(0.75))

# Calculate the Interquartile Range
iqr = df[col].quantile(0.75) - df[col].quantile(0.25)
# print(iqr)
# Calculate the lower bound of the distribution
lower_bound = df[col].quantile(0.25) - (1.5 * iqr)
# print(lower_bound)

print(f"Total number of students whose attendance rate is below the lower bound: {len(df[df[col] < lower_bound]):,}")

### Barchart of `age` column

In [ ]:
# Count the frequency of each unique age using value_counts() to create a Series
age_counts = df['age'].value_counts().sort_index()

# --- 2. Create the Bar Chart from the counts ---
sns.set_style("darkgrid")

plt.figure(figsize=(10, 6))

# Use sns.barplot on the counted data
ax = sns.barplot(x=age_counts.index, y=age_counts.values, palette='viridis')

# --- Add Titles and Labels ---
ax.set_title('Bar Chart of Age Counts')  ##, fontsize=16)
ax.set_xlabel('Age')  ##, fontsize=12)
ax.set_ylabel('Frequency (Count)')  ##, fontsize=12)

# Improve readability of x-axis labels if there are many ages
plt.xticks(rotation=45)  ##, ha='right')
plt.tight_layout() # Adjust layout to make room for rotated labels

plt.savefig(PLOT_PATH + 'barchart')

# Display the plot
plt.show()

The students age are equally balanced for 15 and 16 years. 

### Breakdown of Students' Age by Gender

In [ ]:
# Creating a cross tabular data for age and gender columns
age_gender_crosstab = pd.crosstab(index = df['age'],
                                  columns = df['gender'])

print(age_gender_crosstab)

### Analysis of `hours_per_week`

In [ ]:
df['hours_per_week'].describe()

In [ ]:
# Set a visually appealing style for the plot
sns.set_theme(style="darkgrid")

# Create a figure and axes for the plot. This allows for size adjustments.
plt.figure(figsize=(10, 6))

# Generate the histogram using seaborn's histplot
# 'kde=True' adds a smooth line to estimate the distribution's shape
ax = sns.histplot(data=df, x='hours_per_week', bins=5, kde=True)  ## color='skyblue')

# --- 3. Customizing the Plot ---

# Add a title and labels for clarity and professionalism
ax.set_title('Distribution of Number of Hours Put in to Study') ## , fontsize=18, pad=20)
ax.set_xlabel('Number of Hours')  ##, fontsize=14)
ax.set_ylabel('Number of Students (Frequency)') ##, fontsize=14)

# You can even add a vertical line for the mean score
mean_score = df['hours_per_week'].mean()
ax.axvline(mean_score, color='red', linestyle='--', linewidth=2, label=f'Mean Score: {mean_score:.2f}')
ax.legend() # Display the label for the mean line

plt.savefig(PLOT_PATH + 'histogram_hrs_per_week.png')

# Display the final plot
plt.show()

## Bivariate Analysis

In [ ]:
Logger.debug('Starting Bivariate Analysis.')

### Comparing Sleep against Final Results Attained

In [ ]:
# --- Create and plot the dot plot (scatter plot) ---
plt.figure(figsize=(10, 6)) # Set the figure size

# Create the scatter plot
sns.scatterplot(
    data=df, 
    x='sleep_minutes', 
    y='final_test',
    alpha=0.7, # Make points slightly transparent to see overlaps
    edgecolor='k', # Add a black edge to dots for better visibility
    s=80 # Set the size of the dots
)

# Add enhancements for clarity
plt.title('Final Test Score vs. Sleep Minutes', fontsize=16)
plt.xlabel('Sleep per Night (Minutes)', fontsize=12)
plt.ylabel('Final Test Score', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.show()


### Comparing Attendance Rate against Final Results Attained

In [ ]:
# --- Create and plot the dot plot (scatter plot) ---
plt.figure(figsize=(10, 6)) # Set the figure size

# Create the scatter plot
sns.scatterplot(
    data=df, 
    x='final_test', 
    y='attendance_rate',
    alpha=0.7, # Make points slightly transparent to see overlaps
    edgecolor='k', # Add a black edge to dots for better visibility
    s=80 # Set the size of the dots
)

# Add enhancements for clarity
plt.title('Final Test Score vs. Sleep Minutes', fontsize=16)
plt.xlabel('Sleep per Night (Minutes)', fontsize=12)
plt.ylabel('Final Test Score', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.show()



In [ ]:
# --- Create and plot the dot plot (scatter plot) ---
plt.figure(figsize=(10, 6)) # Set the figure size

# Create the scatter plot
sns.scatterplot(
    data=df, 
    x='sleep_minutes', 
    y='attendance_rate',
    alpha=0.7, # Make points slightly transparent to see overlaps
    edgecolor='k', # Add a black edge to dots for better visibility
    s=80 # Set the size of the dots
)

# Add enhancements for clarity
plt.title('Final Test Score vs. Sleep Minutes', fontsize=16)
plt.xlabel('Sleep per Night (Minutes)', fontsize=12)
plt.ylabel('Final Test Score', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.show()


## Correlation Analysis

In [ ]:
Logger.info('Performing Correlation Analysis.')

### Creating a correlation

In [ ]:
# Create a correlation matrix
corr_matrix = df[['hours_per_week', 'attendance_rate', 'sleep_minutes', 'final_test']].corr()

print("Correlation Matrix:")
print(corr_matrix)

In [ ]:
# Create a correlation matrix
corr_matrix2 = df[['attendance_rate', 'sleep_minutes']].corr()

print("Correlation Matrix:")
print(corr_matrix2)

In [ ]:
# Set the plot style
sns.set_style('darkgrid')

# Plot the correlation map (heatmap)
plt.figure(figsize=(10, 8)) # Set the size of the figure
sns.heatmap(
    corr_matrix, 
    annot=True,          # Annotate the values in the cells
    cmap='coolwarm',     # Choose a color map
    fmt=".2f",           # Format the annotations to two decimal places
    linewidths=.5        # Add lines between cells
)

plt.title('Correlation Map of Student Data', fontsize=16)

plt.savefig(PLOT_PATH + 'heatmap_test_sleep_attendance_study_hrs.png')
plt.show()


From the heatmap above, the features `'sleep_minutes'` and `'attendance_rate'` are highly correlated at 0.87. As such, I will drop the column `'sleep_minutes'` from the model.

#### Dropping the feature `sleep_minutes`

In [ ]:
df.drop(columns = ['sleep_minutes'], inplace = True)
Logger.info('Dropping sleep_minutes column as it highly correlates with attendance_rate column.')

## Feature Engineering

### Feature/Target Separation

In [ ]:
# Separate the feature from the target
X = df.drop(columns = ['final_test'])
y = df['final_test']
Logger.debug('Performed Feature/Target separation.')

In [ ]:
X

In [ ]:
y

### Train-Validation-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and test-validation (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Split the test-validation set (20%) into validation (10%) and test (10%) sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Logger.debug('Performed train/validation/test split.')

In [ ]:
# Display the shapes of the splits to verify
print('Training set shape:\t', X_train.shape, y_train.shape)
Logger.debug(f'Training set shape: {X_train.shape} by {y_train.shape}')

print('Validation set shape:\t', X_val.shape, y_val.shape)
Logger.debug(f'Validation set shape: {X_train.shape} by {y_val.shape}')

print('Test set shape:\t\t', X_test.shape, y_test.shape)
Logger.debug(f'Test set shape: {X_test.shape} by {y_test.shape}')

## Build a Pipeline

In [ ]:
Logger.debug('Building machine learning pipeline.')

### Implementing Standardization

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define numerical features to be standardized
numerical_features = ['n_male', 'n_female', 'hours_per_week', 'attendance_rate']

# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
Logger.debug(f'Preparing numerical_transformer, standardizing numerical features: {numerical_features}.')

### Implementing Feature Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Define nomial features to be one-hot encoded
nominal_features = ['direct_admission', 'CCA', 'learning_style', 'tuition', 'gender', 'bag_color']

# Define passthrough features that will not be transformed
passthrough_features = ['age']

# Create a nominal transformer pipeline
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
Logger.debug(f'Preparing nominal_transformer, performing one-hot encoding on nominal features: {nominal_features}.')
Logger.debug(f'Preparing passthrough feature: {passthrough_features}.')

### Combine Feature Scaling and Encoding

In [ ]:
from sklearn.compose import ColumnTransformer

# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) # Pass through the storey_range feature without transformation
    ],
#     remainder='passthrough',
    n_jobs=-1
    )
Logger.debug('Preparing ColumnTransformer - preprocessor with the numerical, nominal and passthrough transformers.')

In [ ]:
preprocessor

## Model Development

### Implementring Multivariate Linear Regression

In [ ]:
Logger.debug('Implementing Multivariate Linear Regression.')

In [ ]:
from sklearn.linear_model import LinearRegression

# Create the pipeline with a linear regression model
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
Logger.debug('Building a Linear Regression Model with the preprocessor ColumnTransformer object.')

In [ ]:
lr_pipeline

In [ ]:
# Fit the pipeline to the training data
lr_pipeline.fit(X_train, y_train)
Logger.debug('Fitting the training dataset to the Multivariate Linear Regression model - lr_pipeline.')

# Predict on the validation set
y_val_pred = lr_pipeline.predict(X_val)
Logger.debug('Predicting on the validation set - y_val_pred.')

In [ ]:
y_val_pred

## Model Evaluation

### Implementing Model Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score

# Calculate regression metrics for validation set
val_mae = mean_absolute_error(y_val, y_val_pred)
val_mse = mean_squared_error(y_val, y_val_pred)
val_rmse = root_mean_squared_error(y_val, y_val_pred)  # RMSE is the square root of MSE
val_r2 = r2_score(y_val, y_val_pred)

# Display the metrics
Logger.debug('Model metrics of Multivariate Linear Regression.')
print(f"Validation MAE: {val_mae:,.4f}")
Logger.debug(f"Validation MAE: {val_mae:,.4f}")
print(f"Validation MSE: {val_mse:,.4f}")
Logger.debug(f"Validation MSE: {val_mse:,.4f}")
print(f"Validation RMSE: {val_rmse:,.4f}")
Logger.debug(f"Validation RMSE: {val_rmse:,.4f}")
print(f"Validation R²: {val_r2:,.4f}")
Logger.debug(f"Validation R²: {val_r2:,.4f}")

### Implementring Ridge Regression

In [ ]:
Logger.debug('Initating Ridge Regression Model.')
from sklearn.linear_model import Ridge

# Fit Ridge Regression with lambda (alpha in scikit-learn)
ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1))
])
Logger.debug('Building a Ridge Regression Model with the preprocessor ColumnTransformer object.')

ridge_pipeline.fit(X_train, y_train)

Logger.debug('Fitting the training dataset to the Multivariate Linear Regression model - lr_pipeline.')

# Predict on the validation set with Ridge Regression
y_val_pred_ridge = ridge_pipeline.predict(X_val)
Logger.debug('Predicting on the validation set - y_val_pred_ridge.')

# Calculate regression metrics for validation set with Ridge Regression
val_mae_ridge = mean_absolute_error(y_val, y_val_pred_ridge)
val_mse_ridge = mean_squared_error(y_val, y_val_pred_ridge)
val_rmse_ridge = root_mean_squared_error(y_val, y_val_pred_ridge)  # RMSE is the square root of MSE
val_r2_ridge = r2_score(y_val, y_val_pred_ridge)

# Display the metrics for Ridge Regression
print("Ridge Regression Metrics:")
Logger.debug('Model metrics of Ridge Regression.')
print(f"Ridge Validation MAE: {val_mae_ridge:,.4f}")
Logger.debug(f"Validation MAE: {val_mae_ridge:,.4f}")
print(f"Ridge Validation MSE: {val_mse_ridge:,.4f}")
Logger.debug(f"Validation MSE: {val_mse_ridge:,.4f}")
print(f"Ridge Validation RMSE: {val_rmse_ridge:,.4f}")
Logger.debug(f"Validation RMSE: {val_rmse_ridge:,.4f}")
print(f"Ridge Validation R²: {val_r2_ridge:,.4f}")
Logger.debug(f"Validation R²: {val_r2_ridge:,.4f}")

#### Comparing Coefficients

In [ ]:
# Extract the coefficients of the linear regression model
lr_coefs = lr_pipeline.named_steps['regressor'].coef_

# Extract the coefficients of the ridge regression model
ridge_coefs = ridge_pipeline.named_steps['regressor'].coef_

# Assuming feature_names contains the names of the features after preprocessing
feature_names = (
    numerical_features +
    list(ridge_pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(nominal_features)) +
    # ordinal_features +
    passthrough_features
)

# Create a DataFrame for plotting
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Linear Regression': lr_coefs,
    'Ridge Regression': ridge_coefs
})

# Melt the DataFrame to long format for seaborn
coef_df = coef_df.melt(id_vars='Feature', var_name='Model', value_name='Coefficient')

# Plotting the coefficients with seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=coef_df, x='Feature', y='Coefficient', hue='Model', palette=['darkblue', 'lightgreen'])

plt.xlabel('Feature')
plt.ylabel('Coefficient Magnitude')
plt.title('Bar Plot of Linear Regression and Ridge Regression Coefficients')
plt.xticks(rotation=90)
plt.grid(True, axis='y')
plt.legend(loc='upper left')
plt.savefig(PLOT_PATH + 'linear_ridge_regression_coefficients.png')
plt.show()


##### Analysis of the comparison of the Linear Regression and a regularized Ridge Regression



In [ ]:
X_train.columns

In [ ]:
'number_of_siblings', 'direct_admission', 'CCA', 'learning_style', 'gender', 'tuition', 'n_male', 'n_female', 'age', 'hours_per_week', 'attendance_rate', 'mode_of_transport', 'bag_color'

### Implementing Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

# Fit Lasso Regression with default alpha
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=1))
])
lasso_pipeline.fit(X_train, y_train)

# Predict on the validation set with Lasso Regression
y_val_pred_lasso = lasso_pipeline.predict(X_val)

# Calculate regression metrics for validation set with Lasso Regression
val_mae_lasso = mean_absolute_error(y_val, y_val_pred_lasso)
val_mse_lasso = mean_squared_error(y_val, y_val_pred_lasso)
val_rmse_lasso = root_mean_squared_error(y_val, y_val_pred_lasso)  # RMSE is the square root of MSE
val_r2_lasso = r2_score(y_val, y_val_pred_lasso)

# Display the metrics for Lasso Regression
print("Lasso Regression Metrics:")
print(f"Lasso Validation MAE: {val_mae_lasso:,.4f}")
print(f"Lasso Validation MSE: {val_mse_lasso:,.4f}")
print(f"Lasso Validation RMSE: {val_rmse_lasso:,.4f}")
print(f"Lasso Validation R²: {val_r2_lasso:,.4f}")


#### Comparing Coefficients

In [ ]:
# Extract the coefficients of the linear and lasso regression models
lr_coefs = lr_pipeline.named_steps['regressor'].coef_
lasso_coefs = lasso_pipeline.named_steps['regressor'].coef_

# Extract feature names after preprocessing
feature_names = (
    numerical_features +
    list(preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(nominal_features)) +
    # ordinal_features +
    passthrough_features
)

# Create a DataFrame for plotting
coef_df = pd.DataFrame({
    'Feature': feature_names,
    # 'Linear Regression': lr_coefs,
    'Lasso Regression': lasso_coefs
})

# Melt the DataFrame to long format for seaborn
coef_df = coef_df.melt(id_vars='Feature', var_name='Model', value_name='Coefficient')

# Plotting the coefficients with seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=coef_df, x='Feature', y='Coefficient', hue='Model', palette=['darkblue', 'salmon'])

plt.xlabel('Feature')
plt.ylabel('Coefficient Magnitude')
plt.title('Bar Plot of Linear Regression and Lasso Regression Coefficients')
plt.xticks(rotation=90)
plt.grid(True, axis='y')
plt.legend(loc='upper left')
plt.savefig(PLOT_PATH + 'linear_lasso_regression_coefficients.png')
plt.show()

### Feature Selection

From the above graphs, 4 features are showing to have strong impact on the target are, by order of absolute coefficient values:
- CCA_NONE
- n_female
- n_male
- attendance_rate

The following 3 columns have some lower impact on the targer.
- learning_stle_Auditory
- tuition_N
- hours_per_week

The other columns have zero impact and will be omitted from the study.

In [ ]:
X

In [ ]:
# display(lr_coefs)
# display(ridge_coefs)
# display(lasso_coefs)
consol_df_coefs = pd.DataFrame({
    'Feature_name': feature_names,
    'Linear_Regression': lr_coefs,
    'Ridge_Regression': ridge_coefs,
    'Lasso_Regression': lasso_coefs
})
display(consol_df_coefs)
consol_df_coefs.to_csv('./data/consol_df_coefs.csv')

In [ ]:
consol_df_coefs['Lasso_Regression'] = np.abs(consol_df_coefs['Lasso_Regression'])
consol_df_coefs = consol_df_coefs.sort_values(by='Lasso_Regression', ascending=False).reset_index(drop=True)

# --- 4. Select Non-Zero Features ---

# A feature is "selected" if its coefficient is not zero (or very close to zero due to machine precision)
selected_features_df = consol_df_coefs[consol_df_coefs['Lasso_Regression'] != 0].copy()
display(selected_features_df)
# Get a list of the names of the selected features
final_selected_features = selected_features_df['Feature_name'].tolist()
display(final_selected_features)

In [ ]:
lasso_coefs_df = pd.DataFrame({
    'Feature_name': feature_names,
    'Lasso_Regression': lasso_coefs
})
lasso_coefs_df['Lasso_Regression'] = lasso_coefs_df['Lasso_Regression'].abs()
# features = lasso_coefs_df[lasso_coefs_df['Lasso_Regression'] > 0]
# display(lasso_coefs_df[lasso_coefs_df['Lasso_Regression'] > 0])
display(lasso_coefs_df)
# display(features)
# 2. Filter for coefficients greater than 0
# positive_coefs = lasso_coefs_df[lasso_coefs_df['Lasso_Regression'] > 0]
# display(positive_coefs)

The 

### Implementing GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'regressor__alpha': [0.1, 1, 10, 100, 1000],
    'regressor__fit_intercept': [True, False]
}

# Create the pipeline with a ridge regression model
ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Create the pipeline with a lasso regression model
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso())
])

# Perform grid search with cross-validation for Ridge regression
ridge_grid_search = GridSearchCV(ridge_pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
ridge_grid_search.fit(X_train, y_train)

# Perform grid search with cross-validation for Lasso regression
lasso_grid_search = GridSearchCV(lasso_pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
lasso_grid_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters found by grid search for Ridge regression
print("Best Ridge parameters:", ridge_grid_search.best_params_)

# Print the best parameters found by grid search for Lasso regression
print("Best Lasso parameters:", lasso_grid_search.best_params_)

In [ ]:
# Evaluate the best Ridge model on the validation set
best_ridge_model = ridge_grid_search.best_estimator_
y_val_pred_ridge = best_ridge_model.predict(X_val)

# Calculate regression metrics for validation set for Ridge
val_mae_ridge = mean_absolute_error(y_val, y_val_pred_ridge)
val_mse_ridge = mean_squared_error(y_val, y_val_pred_ridge)
val_rmse_ridge = root_mean_squared_error(y_val, y_val_pred_ridge)  # RMSE is the square root of MSE
val_r2_ridge = r2_score(y_val, y_val_pred_ridge)

print("Best Ridge Regression Metrics:")
print(f"Ridge Validation MAE: {val_mae_ridge:.4f}")
print(f"Ridge Validation MSE: {val_mse_ridge:.4f}")
print(f"Ridge Validation RMSE: {val_rmse_ridge:.4f}")
print(f"Ridge Validation R²: {val_r2_ridge:.4f}")
print()

# Evaluate the best Lasso model on the validation set
best_lasso_model = lasso_grid_search.best_estimator_
y_val_pred_lasso = best_lasso_model.predict(X_val)

# Calculate regression metrics for validation set for Lasso
val_mae_lasso = mean_absolute_error(y_val, y_val_pred_lasso)
val_mse_lasso = mean_squared_error(y_val, y_val_pred_lasso)
val_rmse_lasso = root_mean_squared_error(y_val, y_val_pred_lasso)  # RMSE is the square root of MSE
val_r2_lasso = r2_score(y_val, y_val_pred_lasso)

print("Best Lasso Regression Metrics:")
print(f"Lasso Validation MAE: {val_mae_lasso:.4f}")
print(f"Lasso Validation MSE: {val_mse_lasso:.4f}")
print(f"Lasso Validation RMSE: {val_rmse_lasso:.4f}")
print(f"Lasso Validation R²: {val_r2_lasso:.4f}")
print()

In [ ]:
# Print the best parameters found by grid search for Ridge regression
print("Best Ridge parameters:", ridge_grid_search.best_params_)

# Print the best parameters found by grid search for Lasso regression
print("Best Lasso parameters:", lasso_grid_search.best_params_)

In [ ]:
# Evaluate the best Ridge model on the validation set
best_ridge_model = ridge_grid_search.best_estimator_
print(best_ridge_model)
y_val_pred_ridge = best_ridge_model.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set for Ridge
val_mae_ridge = mean_absolute_error(y_val, y_val_pred_ridge)
val_mse_ridge = mean_squared_error(y_val, y_val_pred_ridge)
val_rmse_ridge = root_mean_squared_error(y_val, y_val_pred_ridge)  # RMSE is the square root of MSE
val_r2_ridge = r2_score(y_val, y_val_pred_ridge)

print("Best Ridge Regression Metrics:")
print(f"Ridge Validation MAE: {val_mae_ridge:,.4f}")
print(f"Ridge Validation MSE: {val_mse_ridge:,.4f}")
print(f"Ridge Validation RMSE: {val_rmse_ridge:,.4f}")
print(f"Ridge Validation R²: {val_r2_ridge:,.4f}")
print()

# Evaluate the best Lasso model on the validation set
best_lasso_model = lasso_grid_search.best_estimator_
y_val_pred_lasso = best_lasso_model.predict(X_val)

# Calculate regression metrics for validation set for Lasso
val_mae_lasso = mean_absolute_error(y_val, y_val_pred_lasso)
val_mse_lasso = mean_squared_error(y_val, y_val_pred_lasso)
val_rmse_lasso = root_mean_squared_error(y_val, y_val_pred_lasso)  # RMSE is the square root of MSE
val_r2_lasso = r2_score(y_val, y_val_pred_lasso)

print("Best Lasso Regression Metrics:")
print(f"Lasso Validation MAE: {val_mae_lasso:,.4f}")
print(f"Lasso Validation MSE: {val_mse_lasso:,.4f}")
print(f"Lasso Validation RMSE: {val_rmse_lasso:,.4f}")
print(f"Lasso Validation R²: {val_r2_lasso:,.4f}")
print()

### Implementing RandomSearchCV

In [ ]:
# Best model from Ridge regression (Grid Search)
best_ridge_model = ridge_grid_search.best_estimator_

print(best_lasso_model)

In [ ]:
# Predict on the test set with Ridge Regression
y_test_pred_ridge = best_ridge_model.predict(X_test)

# Calculate regression metrics for the test set for Ridge
test_mae_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
test_mse_ridge = mean_squared_error(y_test, y_test_pred_ridge)
test_rmse_ridge = root_mean_squared_error(y_test, y_test_pred_ridge)
test_r2_ridge = r2_score(y_test, y_test_pred_ridge)

print("Best Ridge Regression Model, Final Test Metrics:")
print(f"Final Test MAE: {test_mae_ridge:,.4f}")
print(f"Final Test MSE: {test_mse_ridge:,.4f}")
print(f"Final Test RMSE: {test_rmse_ridge:,.4f}")
print(f"Final Test R²: {test_r2_ridge:,.4f}")
print()

***End of Notebook***